## 1. Configuración y Carga de Datos

In [ ]:
# Importar librerías
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['figure.dpi'] = 100
sns.set_palette("husl")
sns.set_style("whitegrid")

print("✓ Librerías cargadas correctamente")

In [ ]:
# Rutas de archivos
BASE_DIR = Path.cwd().parent
DATA_VEC = BASE_DIR / 'data' / 'vector'
DATA_PROC = BASE_DIR / 'data' / 'processed'
FIGURES_DIR = BASE_DIR / 'outputs' / 'figures'

# Cargar zonas con estadísticas
gdf_zonas = gpd.read_file(DATA_VEC / 'zonas_con_datos.gpkg')

# Cargar ranking
df_ranking = pd.read_csv(DATA_PROC / 'ranking_zonas.csv')

# Cargar evolución temporal
df_temporal = pd.read_csv(DATA_PROC / 'evolucion_temporal.csv')

print(f"✓ Datos cargados:")
print(f"  - {len(gdf_zonas)} zonas de análisis")
print(f"  - Top {len(df_ranking)} hotspots")
print(f"  - {len(df_temporal)} años analizados")

## 2. Exploración de Resultados Zonales

In [ ]:
# Resumen estadístico
print("═"*70)
print("📊 RESUMEN GLOBAL DE CAMBIOS")
print("═"*70)
print(f"\n🔸 Urbanización:")
print(f"   Total:   {gdf_zonas['urbanizacion_ha'].sum():8.2f} ha")
print(f"   Media:   {gdf_zonas['urbanizacion_ha'].mean():8.2f} ha/zona")
print(f"   Máxima:  {gdf_zonas['urbanizacion_ha'].max():8.2f} ha (zona {gdf_zonas.loc[gdf_zonas['urbanizacion_ha'].idxmax(), 'zona_id']})")

print(f"\n🔸 Pérdida de Vegetación:")
print(f"   Total:   {gdf_zonas['perdida_vegetacion_ha'].sum():8.2f} ha")
print(f"   Media:   {gdf_zonas['perdida_vegetacion_ha'].mean():8.2f} ha/zona")

print(f"\n🔸 Ganancia de Vegetación:")
print(f"   Total:   {gdf_zonas['ganancia_vegetacion_ha'].sum():8.2f} ha")
print(f"   Media:   {gdf_zonas['ganancia_vegetacion_ha'].mean():8.2f} ha/zona")

print(f"\n🔸 Balance Neto:")
balance_veg = gdf_zonas['ganancia_vegetacion_ha'].sum() - gdf_zonas['perdida_vegetacion_ha'].sum()
print(f"   Vegetación: {balance_veg:+8.2f} ha ({balance_veg/gdf_zonas['perdida_vegetacion_ha'].sum()*100:+.1f}%)")

In [ ]:
# Distribución de cambios por zona
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histograma urbanización
axes[0, 0].hist(gdf_zonas['urbanizacion_ha'], bins=20, color='brown', alpha=0.7, edgecolor='black')
axes[0, 0].axvline(gdf_zonas['urbanizacion_ha'].mean(), color='red', linestyle='--', linewidth=2, label='Media')
axes[0, 0].axvline(gdf_zonas['urbanizacion_ha'].median(), color='orange', linestyle='--', linewidth=2, label='Mediana')
axes[0, 0].set_xlabel('Urbanización (ha)', fontweight='bold')
axes[0, 0].set_ylabel('Frecuencia', fontweight='bold')
axes[0, 0].set_title('Distribución de Urbanización por Zona', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Histograma pérdida vegetación
axes[0, 1].hist(gdf_zonas['perdida_vegetacion_ha'], bins=20, color='red', alpha=0.7, edgecolor='black')
axes[0, 1].axvline(gdf_zonas['perdida_vegetacion_ha'].mean(), color='darkred', linestyle='--', linewidth=2, label='Media')
axes[0, 1].set_xlabel('Pérdida Vegetación (ha)', fontweight='bold')
axes[0, 1].set_ylabel('Frecuencia', fontweight='bold')
axes[0, 1].set_title('Distribución de Pérdida de Vegetación', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Histograma ganancia vegetación
axes[1, 0].hist(gdf_zonas['ganancia_vegetacion_ha'], bins=20, color='green', alpha=0.7, edgecolor='black')
axes[1, 0].axvline(gdf_zonas['ganancia_vegetacion_ha'].mean(), color='darkgreen', linestyle='--', linewidth=2, label='Media')
axes[1, 0].set_xlabel('Ganancia Vegetación (ha)', fontweight='bold')
axes[1, 0].set_ylabel('Frecuencia', fontweight='bold')
axes[1, 0].set_title('Distribución de Ganancia de Vegetación', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Box plot comparativo
data_boxplot = [
    gdf_zonas['urbanizacion_ha'],
    gdf_zonas['perdida_vegetacion_ha'],
    gdf_zonas['ganancia_vegetacion_ha']
]
bp = axes[1, 1].boxplot(data_boxplot, labels=['Urbanización', 'Pérdida Veg', 'Ganancia Veg'],
                        patch_artist=True, notch=True)
colors = ['brown', 'red', 'green']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
axes[1, 1].set_ylabel('Hectáreas (ha)', fontweight='bold')
axes[1, 1].set_title('Comparación de Cambios', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.suptitle('Análisis Estadístico de Cambios por Zona - Peñaflor 2018-2024', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## 3. Mapas Coropléticos de Intensidad

In [ ]:
# Mapas coropléticos
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

columnas_mapear = [
    ('urbanizacion_ha', 'Urbanización (ha)', 'YlOrRd'),
    ('perdida_vegetacion_ha', 'Pérdida Vegetación (ha)', 'Reds'),
    ('ganancia_vegetacion_ha', 'Ganancia Vegetación (ha)', 'Greens'),
    ('cambio_total_pct', 'Cambio Total (%)', 'viridis')
]

for idx, (columna, titulo, cmap) in enumerate(columnas_mapear):
    ax = axes[idx]
    
    # Mapa coroplético
    gdf_zonas.plot(
        column=columna,
        cmap=cmap,
        legend=True,
        ax=ax,
        edgecolor='black',
        linewidth=0.5,
        legend_kwds={'label': titulo, 'shrink': 0.8, 'orientation': 'horizontal'}
    )
    
    ax.set_title(titulo, fontsize=14, fontweight='bold')
    ax.set_xlabel('Longitud')
    ax.set_ylabel('Latitud')
    ax.grid(True, alpha=0.3)
    
    # Destacar zonas con valores extremos
    top_zonas = gdf_zonas.nlargest(3, columna)
    for _, zona in top_zonas.iterrows():
        centroid = zona.geometry.centroid
        ax.plot(centroid.x, centroid.y, 'r*', markersize=15)

plt.suptitle('Mapas de Intensidad de Cambio por Zona - Peñaflor 2018-2024', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## 4. Ranking de Hotspots de Urbanización

In [ ]:
# Top 10 zonas con mayor urbanización
print("═"*70)
print("🔥 TOP 10 HOTSPOTS DE URBANIZACIÓN")
print("═"*70)
print()

# Mostrar tabla formateada
df_ranking_display = df_ranking[[
    'zona_id', 
    'urbanizacion_ha', 
    'perdida_vegetacion_ha', 
    'ganancia_vegetacion_ha',
    'indice_transformacion'
]].head(10)

df_ranking_display.columns = [
    'Zona', 
    'Urbanización (ha)', 
    'Pérdida Veg (ha)', 
    'Ganancia Veg (ha)',
    'Índice Transf'
]

# Formatear números
for col in df_ranking_display.columns[1:]:
    df_ranking_display[col] = df_ranking_display[col].apply(lambda x: f"{x:.2f}")

print(df_ranking_display.to_string(index=False))
print()
print(f"💡 Índice de Transformación = Urbanización + Pérdida Veg - Ganancia Veg")
print(f"   Mide el impacto neto de cambio en cada zona")

In [ ]:
# Gráfico de barras horizontal
fig, ax = plt.subplots(figsize=(12, 8))

y_pos = np.arange(len(df_ranking))
urbanizacion = df_ranking['urbanizacion_ha'].values
zonas = df_ranking['zona_id'].values

bars = ax.barh(y_pos, urbanizacion, color='brown', alpha=0.7, edgecolor='black')

# Colorear el top 3
bars[0].set_color('darkred')
bars[1].set_color('red')
bars[2].set_color('orangered')

ax.set_yticks(y_pos)
ax.set_yticklabels(zonas)
ax.invert_yaxis()
ax.set_xlabel('Urbanización (ha)', fontweight='bold', fontsize=12)
ax.set_title('Top 10 Zonas con Mayor Urbanización', fontsize=16, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Agregar valores en las barras
for i, v in enumerate(urbanizacion):
    ax.text(v + 1, i, f'{v:.1f} ha', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 5. Análisis de Evolución Temporal

In [ ]:
# Tabla de evolución temporal
print("═"*70)
print("📅 EVOLUCIÓN TEMPORAL DE ÍNDICES ESPECTRALES")
print("═"*70)
print()

df_temp_display = df_temporal[[
    'fecha', 
    'ndvi_mean', 
    'ndbi_mean', 
    'pct_vegetacion', 
    'pct_urbano'
]].copy()

df_temp_display.columns = ['Año', 'NDVI', 'NDBI', '% Vegetación', '% Urbano']

# Formatear números
df_temp_display['NDVI'] = df_temp_display['NDVI'].apply(lambda x: f"{x:.3f}")
df_temp_display['NDBI'] = df_temp_display['NDBI'].apply(lambda x: f"{x:.3f}")
df_temp_display['% Vegetación'] = df_temp_display['% Vegetación'].apply(lambda x: f"{x:.1f}%")
df_temp_display['% Urbano'] = df_temp_display['% Urbano'].apply(lambda x: f"{x:.1f}%")

print(df_temp_display.to_string(index=False))
print()

# Calcular tendencias
ndvi_change = df_temporal['ndvi_mean'].iloc[-1] - df_temporal['ndvi_mean'].iloc[0]
ndbi_change = df_temporal['ndbi_mean'].iloc[-1] - df_temporal['ndbi_mean'].iloc[0]
urb_change = df_temporal['pct_urbano'].iloc[-1] - df_temporal['pct_urbano'].iloc[0]

print(f"📊 Tendencias 2018-2024:")
print(f"   NDVI:        {ndvi_change:+.3f} ({ndvi_change/df_temporal['ndvi_mean'].iloc[0]*100:+.1f}%)")
print(f"   NDBI:        {ndbi_change:+.3f} ({abs(ndbi_change/df_temporal['ndbi_mean'].iloc[0])*100:+.1f}%)")
print(f"   % Urbano:    {urb_change:+.1f} puntos porcentuales")

In [ ]:
# Gráficos de evolución temporal
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# NDVI temporal
axes[0, 0].errorbar(
    df_temporal['fecha'], 
    df_temporal['ndvi_mean'], 
    yerr=df_temporal['ndvi_std'],
    marker='o', 
    capsize=5, 
    linewidth=2,
    color='green',
    markersize=10
)
axes[0, 0].set_ylabel('NDVI medio ± σ', fontweight='bold')
axes[0, 0].set_title('Evolución del NDVI', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xlabel('Año')

# NDBI temporal
axes[0, 1].errorbar(
    df_temporal['fecha'], 
    df_temporal['ndbi_mean'], 
    yerr=df_temporal['ndbi_std'],
    marker='s', 
    capsize=5, 
    linewidth=2,
    color='brown',
    markersize=10
)
axes[0, 1].set_ylabel('NDBI medio ± σ', fontweight='bold')
axes[0, 1].set_title('Evolución del NDBI', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_xlabel('Año')

# Porcentaje vegetación
axes[1, 0].plot(
    df_temporal['fecha'], 
    df_temporal['pct_vegetacion'], 
    marker='o',
    linewidth=2,
    color='green',
    markersize=10
)
axes[1, 0].fill_between(
    df_temporal['fecha'],
    df_temporal['pct_vegetacion'],
    alpha=0.3,
    color='green'
)
axes[1, 0].set_ylabel('% Área con vegetación', fontweight='bold')
axes[1, 0].set_title('Cobertura de Vegetación', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Año')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Porcentaje urbano
axes[1, 1].plot(
    df_temporal['fecha'], 
    df_temporal['pct_urbano'], 
    marker='s',
    linewidth=2,
    color='gray',
    markersize=10
)
axes[1, 1].fill_between(
    df_temporal['fecha'],
    df_temporal['pct_urbano'],
    alpha=0.3,
    color='gray'
)
axes[1, 1].set_ylabel('% Área urbana', fontweight='bold')
axes[1, 1].set_title('Cobertura Urbana', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Año')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.suptitle('Evolución Temporal de Índices - Peñaflor 2018-2024', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## 6. Análisis de Patrones Espaciales

In [ ]:
# Matriz de correlación entre variables
columnas_correlacion = [
    'urbanizacion_ha',
    'perdida_vegetacion_ha',
    'ganancia_vegetacion_ha',
    'cambio_total_ha',
    'indice_transformacion'
]

corr_matrix = gdf_zonas[columnas_correlacion].corr()

fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(
    corr_matrix, 
    annot=True, 
    fmt='.2f', 
    cmap='coolwarm',
    center=0,
    square=True,
    linewidths=1,
    cbar_kws={"shrink": 0.8},
    ax=ax
)

ax.set_title('Matriz de Correlación entre Variables de Cambio', 
             fontsize=14, fontweight='bold', pad=20)

# Rotar etiquetas para mejor lectura
labels = ['Urbanización', 'Pérdida Veg', 'Ganancia Veg', 'Cambio Total', 'Índice Transf']
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_yticklabels(labels, rotation=0)

plt.tight_layout()
plt.show()

print("\n💡 Interpretación:")
print("   - Valores cercanos a +1: correlación positiva fuerte")
print("   - Valores cercanos a -1: correlación negativa fuerte")
print("   - Valores cercanos a 0: sin correlación")

In [ ]:
# Scatter plot: Urbanización vs Pérdida Vegetación
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Scatter 1: Urbanización vs Pérdida Vegetación
axes[0].scatter(
    gdf_zonas['urbanizacion_ha'],
    gdf_zonas['perdida_vegetacion_ha'],
    c=gdf_zonas['indice_transformacion'],
    cmap='YlOrRd',
    s=100,
    alpha=0.6,
    edgecolors='black'
)
axes[0].set_xlabel('Urbanización (ha)', fontweight='bold')
axes[0].set_ylabel('Pérdida Vegetación (ha)', fontweight='bold')
axes[0].set_title('Urbanización vs Pérdida de Vegetación', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Línea de tendencia
z = np.polyfit(gdf_zonas['urbanizacion_ha'], gdf_zonas['perdida_vegetacion_ha'], 1)
p = np.poly1d(z)
axes[0].plot(
    gdf_zonas['urbanizacion_ha'],
    p(gdf_zonas['urbanizacion_ha']),
    "r--",
    linewidth=2,
    label=f'Tendencia: y={z[0]:.2f}x+{z[1]:.2f}'
)
axes[0].legend()

# Scatter 2: Pérdida vs Ganancia Vegetación
axes[1].scatter(
    gdf_zonas['perdida_vegetacion_ha'],
    gdf_zonas['ganancia_vegetacion_ha'],
    c=gdf_zonas['urbanizacion_ha'],
    cmap='RdYlGn_r',
    s=100,
    alpha=0.6,
    edgecolors='black'
)
axes[1].set_xlabel('Pérdida Vegetación (ha)', fontweight='bold')
axes[1].set_ylabel('Ganancia Vegetación (ha)', fontweight='bold')
axes[1].set_title('Pérdida vs Ganancia de Vegetación', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Línea diagonal de balance
max_val = max(gdf_zonas['perdida_vegetacion_ha'].max(), gdf_zonas['ganancia_vegetacion_ha'].max())
axes[1].plot([0, max_val], [0, max_val], 'k--', linewidth=2, alpha=0.5, label='Balance perfecto')
axes[1].legend()

plt.suptitle('Relaciones entre Variables de Cambio', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 7. Interpretación y Conclusiones

### Hallazgos Clave

1. **Concentración Espacial**: La urbanización no es homogénea; se concentra en ~10 zonas específicas (hotspots) que acumulan >50% del cambio total.

2. **Hotspots Identificados**: Las zonas `Z_02_08`, `Z_07_01`, `Z_06_01` lideran con >60 ha de urbanización cada una, superando 3× la media comunal.

3. **Correlación Urbano-Vegetación**: Existe correlación positiva moderada (r~0.4-0.6) entre urbanización y pérdida de vegetación, indicando que el crecimiento urbano ocurre principalmente sobre áreas vegetadas.

4. **Balance Vegetal Positivo**: A pesar de 2,196 ha de pérdida, se recuperaron 2,463 ha (+12% neto), posiblemente por revegetación de áreas agrícolas abandonadas o programas de restauración.

5. **Tendencia Temporal**: El análisis multitemporal muestra:
   - Pico de urbanización en 2020 (51.1% área urbana)
   - Recuperación vegetativa en 2024 (NDVI +2.8% respecto a 2020)
   - Fluctuaciones interanuales relacionadas con ciclos climáticos (sequía 2019-2022)

### Implicaciones para Planificación Urbana

- **Priorización territorial**: Enfocar intervenciones en hotspots identificados (10 zonas críticas)
- **Instrumentos regulatorios**: Actualizar Plan Regulador con densificación dirigida en zonas consolidadas
- **Mitigación ambiental**: Establecer compensaciones por pérdida de vegetación en zonas >40 ha de cambio
- **Monitoreo continuo**: Implementar seguimiento bianual de zonas críticas

### Limitaciones

- Grilla regular no refleja límites administrativos reales (censos, distritos)
- Resolución temporal limitada (4 fechas, intervalos irregulares)
- No se consideran factores socioeconómicos (precio suelo, inversión pública)

### Próximos Pasos

- **Fase 5**: Desarrollar dashboard interactivo con Streamlit para exploración dinámica de resultados
- **Fase 6**: Generar informe técnico completo con recomendaciones para autoridades locales

---

## Referencias

- **rasterstats**: Perry, M. (2013). Python-rasterstats. https://github.com/perrygeo/python-rasterstats
- **Análisis Zonal**: Esri (2021). *Understanding Zonal Statistics*. ArcGIS Documentation.
- **Hotspot Analysis**: Ord, J. K., & Getis, A. (1995). *Local spatial autocorrelation statistics*. Geographical Analysis, 27(4), 286-306.
- **Planificación Urbana Chile**: Ministerio de Vivienda y Urbanismo (2023). *Plan Regulador Comunal de Peñaflor*.

---

**Fin del Notebook 04**